In [1]:
import matplotlib.pyplot as plt
from envs.env_creator import env_creator
from base_rl.eval_policy import EvalDiscreteStatePolicy
from glob import glob
import pandas as pd
import dill
from benchmarks.policy_benchmarks import PolicyBenchmarksParallel

In [2]:
plt.rcParams["figure.figsize"] = [20, 12]

In [3]:
device = 'cuda'
steps_per_episode = 1000
model_names = ['model_aeq-20bits3']#['model_aeq-20bits3', 'model_h_c-20bits3']
experiment_id = '*'
experiment_type = 'policy_iteration'

In [4]:
model_files = {model_name: glob(f'tmp/{experiment_type}/{experiment_id}/{model_name}/**/*policy.pkl',recursive=True) for model_name in model_names}

In [5]:
def load(save_path):
        with open(save_path, 'rb') as f:
            return dill.load(f)

In [6]:


benchmarks = {}
evaluators = []
for model_name,policy_files in model_files.items():
    for policy_file in policy_files:
        eval_policy = load(policy_file)
        env_kwargs = {'steps_per_episode': steps_per_episode, 'device': device,
                      'model_path': f'tmp/state_quantization/{model_name}'}
        evaluator = EvalDiscreteStatePolicy(policy=eval_policy, env_creator=env_creator, env_kwargs=env_kwargs, tag=policy_file.split('.')[0])
        evaluators.append(evaluator)

policy_benchmarks = PolicyBenchmarksParallel(evaluators=evaluators, epochs=10)
policy_benchmarks.benchmark()

Tag: tmp/policy_iteration/711549/model_aeq-20bits3/10000/policy
Episode 0 Reward: -418.6743063023979 || New State Found: True


Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Process ForkPoolWorker-1:


KeyboardInterrupt: 

In [7]:
df = pd.DataFrame(policy_benchmarks.benchmark_metrics)
df.T

,reward,std,unique_obs
tmp/policy_iteration/977283/model_aeq-20bits3/10/policy,-605.712084,31.105069,284.0
tmp/policy_iteration/977283/model_aeq-20bits3/10000/policy,-343.909135,85.947031,719.0
tmp/policy_iteration/977283/model_aeq-20bits3/1000/policy,-306.250813,87.061688,721.0
tmp/policy_iteration/977283/model_aeq-20bits3/100/policy,-846.507480,28.458800,238.0
tmp/policy_iteration/411975/model_aeq-20bits3/10/policy,-600.938480,38.108825,281.0
tmp/policy_iteration/411975/model_aeq-20bits3/10000/policy,-417.723294,113.880609,782.0
tmp/policy_iteration/411975/model_aeq-20bits3/1000/policy,-272.694446,69.133524,731.0
tmp/policy_iteration/411975/model_aeq-20bits3/100/policy,-826.239242,57.452773,287.0


In [ ]:
model_files

In [ ]:
policy_benchmarks.evaluators[0].eval_trajectories